## Regional SAT anomalies calculation then calculate the trend

In [ ]:
# In[1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %%
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprocess

In [ ]:
# import src.slurm_cluster as scluster
# client, scluster = scluster.init_dask_slurm_cluster(scale=2,cores=10, memory="200GB")

In [ ]:
def func_mk(x):
    """
    Mann-Kendall test for trend
    """
    results = data_process.mk_test(x)
    slope = results[0]
    p_val = results[1]
    return slope, p_val

In [ ]:
# load data
dir_HadCRUT5 = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Supp_Figure1/'
HadCRUT5 = xr.open_dataset(dir_HadCRUT5 + 'tas_HadCRUT5_annual_anomalies_processed.nc')

dir1 ='/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Supplementary/S1/data_revise/'
HadCRUT5_forced = xr.open_mfdataset(dir1 + 'HadCRUT_Forced_signal.nc',chunks={'lat':10,'lon':10})
HadCRUT5_internal = xr.open_mfdataset(dir1 + 'HadCRUT_residual.nc',chunks={'lat':10,'lon':10})

In [ ]:
HadCRUT5

In [ ]:
HadCRUT5 = HadCRUT5.rename({'__xarray_dataarray_variable__':'tas'})
HadCRUT5_forced = HadCRUT5_forced.rename({'__xarray_dataarray_variable__':'tas'})
HadCRUT5_internal = HadCRUT5_internal.rename({'__xarray_dataarray_variable__':'tas'})

In [ ]:
HadCRUT5

### Calculate the trend end year fix to 2022, start with 73 year length and decrease length of trend every one year, the minimum trend length is 10yr 

In [ ]:
temp_data = HadCRUT5.tas
temp_data_forced = HadCRUT5_forced.tas
temp_data_internal = HadCRUT5_internal.tas

In [ ]:
#Perfrom land sea mask
land_sea_mask=xr.open_dataset('/work/mh0033/m301036/Data_storage/CMIP6-MPI-ESM-LR/GR15_lsm_regrid.nc')
# land_sea_mask.coords
land_sea_mask

# mask the land area trend data
mask_data = land_sea_mask['var1']
mask_data

In [ ]:
# Apply the land-sea mask to the trend data, mask the land area trend
temp_data_masked = temp_data.where(mask_data[0,:,:]==0, drop = False)
temp_data_forced_masked = temp_data_forced.where(mask_data[0,:,:]==0, drop = False)
temp_data_internal_masked = temp_data_internal.where(mask_data[0,:,:]==0, drop = False)

In [ ]:
temp_data

In [ ]:
temp_data_masked

In [ ]:
# flip the longitude to -180 to 180
temp_data_adj = preprocess.convert_longitude(temp_data)
temp_data_forced_adj = preprocess.convert_longitude(temp_data_forced)
temp_data_internal_adj = preprocess.convert_longitude(temp_data_internal)

In [ ]:
temp_data_masked_adj = preprocess.convert_longitude(temp_data_masked)
temp_data_forced_masked_adj = preprocess.convert_longitude(temp_data_forced_masked)
temp_data_internal_masked_adj = preprocess.convert_longitude(temp_data_internal_masked)

In [ ]:
temp_data_adj

In [ ]:
type(temp_data_adj)

### Regional anomalies calculation

In [ ]:
def plot_trend(temp_data, lats, lons, levels=None, extend=None, cmap=None, 
                                 title="", ax=None, show_xticks=False, show_yticks=False):
    """
    Plot the trend spatial pattern using Robinson projection with significance overlaid.

    Parameters:
    - temp_data: 2D numpy array with the trend values.
    - lats, lons: 1D arrays of latitudes and longitudes.
    - p_values: 2D array with p-values for each grid point.
    - GMST_p_values: 2D array with GMST p-values for each grid point.
    - title: Title for the plot.
    - ax: Existing axis to plot on. If None, a new axis will be created.
    - show_xticks, show_yticks: Boolean flags to show x and y axis ticks.
    
    Returns:
    - contour_obj: The contour object from the plot.
    """
    # Plotting
    contour_obj = ax.contourf(lons, lats, temp_data, levels=levels, extend=extend, cmap=cmap, transform=ccrs.PlateCarree())

    ax.coastlines(resolution='110m')
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,
                      color='gray', alpha=0.35, linestyle='--')

    # Disable labels on the top and right of the plot
    gl.top_labels = False
    gl.right_labels = False

    # Enable labels on the bottom and left of the plot
    gl.bottom_labels = show_xticks
    gl.left_labels = show_yticks
    gl.xformatter = cticker.LongitudeFormatter()
    gl.yformatter = cticker.LatitudeFormatter()
    gl.xlabel_style = {'size': 16}
    gl.ylabel_style = {'size': 16}
    
    if show_xticks:
        gl.bottom_labels = True
    if show_yticks:
        gl.left_labels = True
    
    # ax.set_title(title, loc='center', fontsize=18, pad=5.0)

    return contour_obj
# %%
plt.rcParams['figure.figsize'] = (8, 10)
plt.rcParams['font.size'] = 16
# plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['ytick.major.right'] = True
plt.rcParams['ytick.right'] = True
plt.rcParams['xtick.bottom'] = True
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'
plt.rcParams['savefig.pad_inches'] = 0.1
plt.rcParams['savefig.transparent'] = True

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import seaborn as sns
from matplotlib.colors import ListedColormap
from matplotlib.colors import BoundaryNorm, ListedColormap
import cartopy.util as cutil
import seaborn as sns
import matplotlib.colors as mcolors
import palettable

In [ ]:
levels = np.arange(-3.5, 4.0, 0.25)

In [ ]:
# Plot the North Atlantic region's temperature anomalies map
region_sel = temp_data_masked_adj.sel(year=2020)
region_sel

fig = plt.figure(figsize=(15, 10))
gs = gridspec.GridSpec(1, 1, figure=fig, hspace=0.01, wspace=0.01)

ax = fig.add_subplot(gs[0,0], projection=ccrs.PlateCarree(central_longitude=0))
# ax.set_global()
# pvalue_annual_da_with_cyclic, _ = cutil.add_cyclic_point(pvalue_annual_da['40yr'], coord=lon)
contour_obj = plot_trend(region_sel, region_sel.lat, region_sel.lon, levels=levels,extend='both', cmap='twilight_shifted',
            title=" ", ax=ax, show_xticks=True, show_yticks=True)
ax.set_title('masked_unforced_SAT', fontsize=22, pad=10.0)

plt.show()

In [ ]:
lat = temp_data_adj.lat
lon = temp_data_adj.lon

lat1 = 42
lat2 = 60
lon1 = -50
lon2 = -10

In [ ]:
def calc_North_Atlantic_anomalies(data,mask_data):
    ds_WH = data.sel(lat=slice(42, 60), lon=slice(-50, -10))
    ds_WH_anomaly = data_process.calc_weighted_mean(ds_WH)
    ds_sel = mask_data.sel(lat=slice(0, 90), lon=slice(-180,180))
    ds_sel_anomaly = data_process.calc_weighted_mean(ds_sel)
    
    ds_anomalies = ds_WH_anomaly - ds_sel_anomaly
    return ds_anomalies

In [ ]:
temp_data_adj_subpolar_gyre = calc_North_Atlantic_anomalies(temp_data_adj,temp_data_masked_adj)

temp_forced_data_adj_subpolar_gyre = calc_North_Atlantic_anomalies(temp_data_forced_adj,temp_data_forced_masked_adj)

temp_internal_data_adj_subpolar_gyre = calc_North_Atlantic_anomalies(temp_data_internal_adj,temp_data_internal_masked_adj)

In [ ]:
temp_data_adj_subpolar_gyre 

In [ ]:
# plot the aractic region anomaly year 2020
# da_plot_subpolar_gyre = temp_internal_da_subpolar_gyre.sel(year=2020)
da_plot = temp_data_internal_adj.sel(year=2020).sel(lat=slice(lat1, lat2), lon=slice(lon1, lon2))

fig, ax = plt.subplots(subplot_kw={'projection': ccrs.Robinson(central_longitude=180)})

contour_obj =  plot_trend(da_plot, da_plot.lat, da_plot.lon, levels=np.arange(-5, 5.5, 0.5), extend='both', cmap='RdBu_r',
                                    title="subpolar_gyre region SAT anomaly 2020", ax=ax, show_xticks=True, show_yticks=True)

# colorbar
cbar = plt.colorbar(contour_obj, ax=ax, orientation='horizontal', pad=0.05, aspect=50)
cbar.set_label('Temperature anomaly (°C)')
cbar.ax.tick_params(labelsize=14)

plt.show()


In [ ]:
dir_out = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure4/data/NAT_wrt_NAT_SST/'

# temp_data_adj_subpolar_gyre.to_netcdf(dir_out + '1850_2022_Raw_subpolar_gyre_mean.nc')
# temp_forced_data_adj_subpolar_gyre.to_netcdf(dir_out + '1850_2022_Forced_subpolar_gyre_mean.nc')
# temp_internal_data_adj_subpolar_gyre.to_netcdf(dir_out + '1850_2022_Internal_subpolar_gyre_mean.nc')

### Segments definitions

In [ ]:
import pandas as pd
import numpy as np

start_year = 1950
end_year = 2022
min_length = 10

In [ ]:
years = np.arange(start_year, end_year + 1)

# Prepare a dictionary to hold the trend DataArrays for each period
temp_data_segments = {}

for begin_year in range(start_year, end_year - min_length+2):
    # Select the data from the start year to the last year
    time_slice = temp_data_adj_subpolar_gyre.sel(year=slice(begin_year, end_year))
    
    temp_data_segments[begin_year] = time_slice

In [ ]:
temp_data_forced_segments = {}
temp_data_internal_segments = {}

for begin_year in range(start_year, end_year - min_length+2):
    # Select the data from the start year to the last year
    time_slice = temp_forced_data_adj_subpolar_gyre.sel(year=slice(begin_year, end_year))
    
    temp_data_forced_segments[begin_year] = time_slice

for begin_year in range(start_year, end_year - min_length+2):
    # Select the data from the start year to the last year
    time_slice = temp_internal_data_adj_subpolar_gyre.sel(year=slice(begin_year, end_year))
    
    temp_data_internal_segments[begin_year] = time_slice

In [ ]:
temp_data_segments

In [ ]:
len(temp_data_segments.keys())

In [ ]:
# calculate the trend for each segment
trends = {}

for begin_year in temp_data_segments.keys():
    temp_data = temp_data_segments[begin_year]
    temp_data_forced = temp_data_forced_segments[begin_year]
    temp_data_internal = temp_data_internal_segments[begin_year]
    
    # Calculate the trend for each segment
    trends[begin_year] = {}
    trends[begin_year]['raw'] = data_process.mk_test(temp_data,a=0.05)[0]*10.0
    trends[begin_year]['forced'] = data_process.mk_test(temp_data_forced,a=0.05)[0]*10.0
    trends[begin_year]['internal'] = data_process.mk_test(temp_data_internal,a=0.05)[0]*10.0

In [ ]:
trends

In [ ]:
# transform the dictionary to dataframe
trends_df = pd.DataFrame(trends).T
trends_df

# for interval, data in temp_annual_np.items():
#     temp_annual_da[interval] = xr.DataArray(data, dims=["lat", "lon"], coords={"lat": temp_data[interval].lat, "lon": temp_data[interval].lon})
# for interval, data in pvalue_annual_np.items():
#     pvalue_annual_da[interval] = xr.DataArray(data, dims=["lat", "lon"], coords={"lat": temp_data[interval].lat, "lon": temp_data[interval].lon})

In [ ]:
# data frame to dataset
trends_ds = trends_df.to_xarray()

In [ ]:
trends_ds

In [ ]:
# export the trend to the netcdf file
trends_ds.to_netcdf(dir_out + 'subpolar_gyre_wrt_NAT_trend_variations.nc')

In [ ]:
dirin = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure4/data/NAT_wrt_nh_Keil/'
subpolar_gyre_unforced_lower = xr.open_dataset(f'{dirin}internal_subpolar_gyre_trend_lower_percentile.nc')
subpolar_gyre_unforced_upper = xr.open_dataset(f'{dirin}internal_subpolar_gyre_trend_upper_percentile.nc')

In [ ]:
subpolar_gyre_unforced_lower

In [ ]:
subpolar_gyre_unforced_lower = subpolar_gyre_unforced_lower.rename_vars({'__xarray_dataarray_variable__':'trend'})
subpolar_gyre_unforced_upper = subpolar_gyre_unforced_upper.rename_vars({'__xarray_dataarray_variable__':'trend'})